## Working with Missing and Duplicate Data 

In this mission, we'll explore each of these options in detail and learn when to use them. We'll work with the 2015, 2016, and 2017 World Happiness Reports again - more specifically, we'll combine them and clean missing values as we start to define a more complete data cleaning workflow.
<br>
<br>
First is to go through the information for the dataset

In [2]:
import pandas as pd
happiness2015 = pd.read_csv("World_Happiness_2015.csv")
happiness2016 = pd.read_csv("World_Happiness_2016.csv")
happiness2017 = pd.read_csv("World_Happiness_2017.csv")

happiness2015['Year'] = 2015
happiness2016['Year'] = 2016
happiness2017['Year'] = 2017

In [3]:
shape_2015 = happiness2015.shape
shape_2016 = happiness2016.shape
shape_2017 = happiness2017.shape

print(shape_2015)
print(shape_2016)
print(shape_2017)

(158, 13)
(157, 14)
(155, 13)


## Identifying Missing Values 

Recall that the dataframes were updated so that each contains the same countries, even if the happiness score, happiness rank, etc. were missing. However, that also means that each likely contains missing values.
<br>
<br>
However, pandas will not automatically identify values such as n/a, -, or -- as NaN or None, but they may also indicate data is missing.
See here for more information on how to use the pd.read_csv() function to read those values in as NaN.
<br>
<br>
Once we ensure that all missing values were read in correctly, we can use the Series.isnull() method to identify rows with missing values

In [4]:
missing_2016 = happiness2016.isnull().sum()
missing_2017 = happiness2017.isnull().sum()
print(missing_2016)
print(missing_2017)

Country                          0
Region                           0
Happiness Rank                   0
Happiness Score                  0
Lower Confidence Interval        0
Upper Confidence Interval        0
Economy (GDP per Capita)         0
Family                           0
Health (Life Expectancy)         0
Freedom                          0
Trust (Government Corruption)    0
Generosity                       0
Dystopia Residual                0
Year                             0
dtype: int64
Country                          0
Happiness.Rank                   0
Happiness.Score                  0
Whisker.high                     0
Whisker.low                      0
Economy..GDP.per.Capita.         0
Family                           0
Health..Life.Expectancy.         0
Freedom                          0
Generosity                       0
Trust..Government.Corruption.    0
Dystopia.Residual                0
Year                             0
dtype: int64


## Correcting Data Cleaning Errors that results in missing values 

It's good to check for missing values before transforming data to make sure we don't unintentionally introduce missing values.
<br>
<br>
If we do introduce missing values after transforming data, we'll have to determine if the data is really missing or if it's the result of some kind of error. As we progress through this mission, we'll use the following workflow to clean our missing values, starting with checking for errors:
- Check for errors in data cleaning/transformation
- Use data from additional sources to fill missing values
- Drop row/column
- Fill missing values with reasonable estimates computed from the available data

In [5]:
combined = pd.concat([happiness2015, happiness2016, happiness2017], ignore_index=True)
print(combined.head())

       Country  Dystopia Residual  Dystopia.Residual  \
0  Switzerland            2.51738                NaN   
1      Iceland            2.70201                NaN   
2      Denmark            2.49204                NaN   
3       Norway            2.46531                NaN   
4       Canada            2.45176                NaN   

   Economy (GDP per Capita)  Economy..GDP.per.Capita.   Family  Freedom  \
0                   1.39651                       NaN  1.34951  0.66557   
1                   1.30232                       NaN  1.40223  0.62877   
2                   1.32548                       NaN  1.36058  0.64938   
3                   1.45900                       NaN  1.33095  0.66973   
4                   1.32629                       NaN  1.32261  0.63297   

   Generosity  Happiness Rank  Happiness Score  ...  Health..Life.Expectancy.  \
0     0.29678             1.0            7.587  ...                       NaN   
1     0.43630             2.0            7.561  ..

C:\Users\pakhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [6]:
combined.isnull().sum()

Country                            0
Dystopia Residual                155
Dystopia.Residual                315
Economy (GDP per Capita)         155
Economy..GDP.per.Capita.         315
Family                             0
Freedom                            0
Generosity                         0
Happiness Rank                   155
Happiness Score                  155
Happiness.Rank                   315
Happiness.Score                  315
Health (Life Expectancy)         155
Health..Life.Expectancy.         315
Lower Confidence Interval        313
Region                           155
Standard Error                   312
Trust (Government Corruption)    155
Trust..Government.Corruption.    315
Upper Confidence Interval        313
Whisker.high                     315
Whisker.low                      315
Year                               0
dtype: int64

we'll update the column names to make them uniform and combine the dataframes again with the replace function. 

In [7]:
happiness2017.columns = happiness2017.columns.str.replace('.', ' ').str.replace('\s+', ' ').str.strip().str.upper()
happiness2015.columns = happiness2015.columns.str.replace(r'[\(\)]', '').str.strip().str.upper()
happiness2016.columns = happiness2016.columns.str.replace(r'[\(\)]', '').str.strip().str.upper()

combined = pd.concat([happiness2015, happiness2016, happiness2017], ignore_index=True)
missing = combined.isnull().sum()
missing

C:\Users\pakhl\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


COUNTRY                          0
DYSTOPIA RESIDUAL                0
ECONOMY GDP PER CAPITA           0
FAMILY                           0
FREEDOM                          0
GENEROSITY                       0
HAPPINESS RANK                   0
HAPPINESS SCORE                  0
HEALTH LIFE EXPECTANCY           0
LOWER CONFIDENCE INTERVAL      313
REGION                         155
STANDARD ERROR                 312
TRUST GOVERNMENT CORRUPTION      0
UPPER CONFIDENCE INTERVAL      313
WHISKER HIGH                   315
WHISKER LOW                    315
YEAR                             0
dtype: int64

And we can pick some missing values for further investigation: 

In [8]:
regions_2017 = combined[combined['YEAR']==2017]['REGION']
missing = regions_2017.isnull().sum()
missing

155

## Using Data From Additional Source to Fill in Missing Value

And we can pick some missing values for further investigation: 

In [9]:
regions_2017 = combined[combined['YEAR']==2017]['REGION']
missing = regions_2017.isnull().sum()
missing

155

We confirmed that the REGION column is missing from the 2017 data. Since we need the regions to analyze our data, let's turn our attention there next.
<br>
<br>
Before we drop or replace any values, let's first see if there's a way we can use other available data to correct the values.
- Check for errors in data cleaning/transformation.
- Use data from additional sources to fill missing values.
- Drop row/column
- Fill missing values with reasonable estimates computed from the available data

Recall once more that each year contains the same countries. Since the regions are fixed values - the region a country was assigned to in 2015 or 2016 won't change - we should be able to assign the 2015 or 2016 region to the 2017 row.

In [10]:
regions = combined[(combined['YEAR']==2015)|(combined['YEAR']==2016)][['COUNTRY', 'REGION']]
print(regions.head(5))

       COUNTRY          REGION
0  Switzerland  Western Europe
1      Iceland  Western Europe
2      Denmark  Western Europe
3       Norway  Western Europe
4       Canada   North America


In [11]:
combined = pd.merge(left=combined, right=regions, on='COUNTRY', how='left')
combined = combined.drop('REGION_x', axis=1)
missing = combined.isnull().sum()
missing

COUNTRY                          0
DYSTOPIA RESIDUAL                0
ECONOMY GDP PER CAPITA           0
FAMILY                           0
FREEDOM                          0
GENEROSITY                       0
HAPPINESS RANK                   0
HAPPINESS SCORE                  0
HEALTH LIFE EXPECTANCY           0
LOWER CONFIDENCE INTERVAL      610
STANDARD ERROR                 609
TRUST GOVERNMENT CORRUPTION      0
UPPER CONFIDENCE INTERVAL      610
WHISKER HIGH                   617
WHISKER LOW                    617
YEAR                             0
REGION_y                         2
dtype: int64

## Identifying Duplicate Value

Before we decide how to handle the rest of our missing values, let's first check our dataframe for duplicate rows.
<br>
<br>
We'll use the DataFrame.duplicated() method to check for duplicate values. If no parameters are specified, the method will check for any rows in which all columns have the same values.
<br>
<br>
However, one thing to keep in mind is that the df.duplicated() method will only look for exact matches, so if the capitalization for country names isn't exactly the same, they won't be identified as duplicates.
<br>
<br>
Since we should only have one country for each year, we can be a little more thorough by defining rows with ONLY the same country and year as duplicates. To accomplish this, let's pass a list of the COUNTRY and YEAR column names into the df.duplicated() method

In [12]:
combined['COUNTRY'] = combined['COUNTRY'].str.upper()
dups = combined.duplicated(['COUNTRY', 'YEAR'])
print(dups)

0      False
1       True
2      False
3       True
4      False
       ...  
913    False
914     True
915    False
916     True
917    False
Length: 918, dtype: bool


## Correcting Duplicates Values 

In the previous screen, we standardized the capitalization of the values in the COUNTRY column and identified that we actually do have three duplicate rows:
<br>
<br>
Let's inspect all the rows for SOMALILAND REGION in combined

In [15]:
combined[combined['COUNTRY'] == 'SOMALILAND REGION']

,COUNTRY,DYSTOPIA RESIDUAL,ECONOMY GDP PER CAPITA,FAMILY,FREEDOM,GENEROSITY,HAPPINESS RANK,HAPPINESS SCORE,HEALTH LIFE EXPECTANCY,LOWER CONFIDENCE INTERVAL,STANDARD ERROR,TRUST GOVERNMENT CORRUPTION,UPPER CONFIDENCE INTERVAL,WHISKER HIGH,WHISKER LOW,YEAR,REGION_y
179,SOMALILAND REGION,2.11032,0.18847,0.95152,0.46582,0.50318,91,5.057,0.43873,NaN,0.06161,0.39928,NaN,NaN,NaN,2015,Sub-Saharan Africa
498,SOMALILAND REGION,2.43801,0.25558,0.75862,0.39130,0.51479,97,5.057,0.33108,4.934,NaN,0.36794,5.18,NaN,NaN,2016,Sub-Saharan Africa


We assume that we shoukd leep the first row:

In [14]:
combined['COUNTRY'] = combined['COUNTRY'].str.upper()
combined = combined.drop_duplicates(['COUNTRY', 'YEAR'], keep='first')
print(combined.head(5))

       COUNTRY  DYSTOPIA RESIDUAL  ECONOMY GDP PER CAPITA   FAMILY  FREEDOM  \
0  SWITZERLAND            2.51738                 1.39651  1.34951  0.66557   
2      ICELAND            2.70201                 1.30232  1.40223  0.62877   
4      DENMARK            2.49204                 1.32548  1.36058  0.64938   
6       NORWAY            2.46531                 1.45900  1.33095  0.66973   
8       CANADA            2.45176                 1.32629  1.32261  0.63297   

   GENEROSITY  HAPPINESS RANK  HAPPINESS SCORE  HEALTH LIFE EXPECTANCY  \
0     0.29678               1            7.587                 0.94143   
2     0.43630               2            7.561                 0.94784   
4     0.34139               3            7.527                 0.87464   
6     0.34699               4            7.522                 0.88521   
8     0.45811               5            7.427                 0.90563   

   LOWER CONFIDENCE INTERVAL  STANDARD ERROR  TRUST GOVERNMENT CORRUPTION  \
0  

## Handle Missing Values by Dropping Columns 

Besides filling the information, another alternative for missing data is to drop them.
<br>
<br>
From the result above, we can see that some columns have too many missing value that it should not be used. In this case, we should drop it

In [18]:
columns_to_drop = ['LOWER CONFIDENCE INTERVAL', 'STANDARD ERROR', 'UPPER CONFIDENCE INTERVAL', 'WHISKER HIGH', 'WHISKER LOW']

combined = combined.drop(columns_to_drop, axis=1)
missing = combined.isnull().sum()

KeyError: "['LOWER CONFIDENCE INTERVAL' 'STANDARD ERROR' 'UPPER CONFIDENCE INTERVAL'\n 'WHISKER HIGH' 'WHISKER LOW'] not found in axis"

The result is then deleted 
<br>
<br>
As you start working with bigger datasets, it can sometimes be tedious to create a long list of column names to drop. Instead we can use the DataFrame.dropna() method to complete the same task.

By default, the dropna() method will drop rows with any missing values. To drop columns, we can set the axis parameter equal to 1, just like with the df.drop() method:

In [20]:
df.dropna(axis=1)

NameError: name 'df' is not defined

## Handle Missing Values by Dropping Columns 

After checking the information, now most of the missing values are fixed.
<br>
<br>
Another alternative to tackle the missing value is imputation i.e ill missing values with reasonable estimates computed from the available data.
<br>
<br>
There are many options for choosing the replacement value, including:
<br>
<br>
A constant value
The mean of the column
The median of the column
The mode of the column
First, let's build some intuition around this technique by analyzing how replacing missing values with the mean affects the distribution of the data. In order to do so, we'll use the Series.fillna() method to replace the missing values with the mean.
<br>
<br>
Note that we must pass the replacement value into the Series.fillna() method. For example, if we wanted to replace all of the missing values in the HAPPINESS SCORE column with 0, we'd use the following syntax:

In [21]:
happiness_mean = combined['HAPPINESS SCORE'].mean()
combined['HAPPINESS SCORE UPDATED'] = combined['HAPPINESS SCORE'].fillna(happiness_mean)
print(combined['HAPPINESS SCORE UPDATED'].mean)

<bound method Series.mean of 0      7.587
2      7.561
4      7.527
6      7.522
8      7.427
       ...  
909    3.471
911    3.462
913    3.349
915    2.905
917    2.693
Name: HAPPINESS SCORE UPDATED, Length: 470, dtype: float64>
